# Shared Interactors Pipeline

### This pipeline takes as input user-defined groups of phospho-peptides (e.g. those with increased phosphorylation in response stimulus or those with decreased phosphorylation in response to stimulus). The first script partitions each group into ‘modules’ of phospho-peptides that share the same sequence motif around the phosphorylation site. The method then identifies Shared Interactors as described above for each identified module. 

# Identify motifs
    
    This automates submitting jobs to the Motif-x Website (http://motif-x.med.harvard.edu/)
    
### Input  : single text file (called inputfiles in next cell) listing excel files to process, one excel file name per line.

    text file:
    data_sheet1.xlsx
    data_sheet2.xlsx

	The Excel file format:
	Ppep	Group	Localized_Sequence	Motif_X_Input_Peptide
	YGL076C_T8_S11	Induced	AAEKILtPEsQLKK	AAEKILT*PES*QLKK

	Column order is unimportant, column names must match above.
    
### Output:
A final text table named after the input file containing all the motifs matched to a gene.<br>
        
Given an input file named,  motifx_sample.xlsx the final results file will be:<br>
    
 > motifx_sample-Motifx-results.txt
         
The rest of the results are put in a directory.  For instance if your input file is called
motifx_sample.xlsx, 3 directories will be created one for each central character.  These 
contain the LOGO pngs and the original html results page.<br>         
 > motifx_sample_T<br>
 > motifx_sample_S<br>
 > motifx_sample_Y<br>

In [ ]:
# load required libraries
import numpy as np
import os
import pandas as pd
import re
from scipy.stats import hypergeom


In [ ]:
%run -i 'Motifx.py' -f 'inputfiles' -u 'reference/orf_trans_all.20150113.fasta'

# Identify Modules and Submodules step.

This script identifies co-regulated groups of phospho-peptides using the following approach:

1) First, the script identifies 'modules', which are groups of phospho-peptides that exhibit the same directionality in stress-dependent abundance change (ie, increased 'Induced', or decreased 'Repressed') and the same motif. The module nomenclature is as follows: Induced/Repressed- motif (ex: Induced..RK.s....).

2) Next, the script partitions modules into 'submodules' based on their phospho-peptide constituents dependency on a protein(s) for stress-dependent abundance changes (ie, phospho-peptides that exhibit increased 'amplified' or decreased 'defective' abundance in a deletion strain compared to the 'WT' or 'Parental' type strain). These phenotypes are user defined. If two or more mutant phenotypes are recorded for a phospho-peptide then it's placed into two separate subModules (one for each mutant phenotype). If there was not a mutant phenotype at a user defined threshold then the phenotype is 'No-Phenotype'

The submodule nomenclature is as follows: module name-mutant phenotype/No-Phenotype (ex: Induced..RK.s....Mutant_Defective).

Possible submodule phenotypes: Induced-Defective, Induced-Amplified, Repressed-Defective, Repressed-Amplified, Induced-No-Phenotype, Repressed-No-Phenotype


The motifx results file need to be classified by the user.  For example the output from the motifx results looks like:
>YNR051C_T336,SGNNASTPSSSPE,InputFileName,......TP.....<br>
>YAL035W_T388,AATPAATPTPSSA,InputFileName,......TP.....<br>
>YML008C_T373,KPENAETPSQTSQ,InputFileName,......TP.....<br>
>YKL204W_T391,KESRSSTPNAESQ,InputFileName,......TP.....<br>

The user must add the grouping. Here is an example of the idModules.csv file used in the next step. 

Ppep,Cluster,Motif,Peptide,ire1,mkk1_2<br>
YGR240C_S895,Induced,......SP.....,NKKNEASPNTDAK,Induced_Amplified,Induced_Defective
YMR005W_S80,Induced,...K..SP.....,VLPKNVSPTTNLR,Induced_Amplified,Induced_Defective
YPL242C_S7,Induced,......SP.....,MTAYSGSPSKPGN,Induced_Amplified, 

Where ire1 and mkk1_2 correspond to gene names and Induced_Amplified,Induced_Defective are groupings provided by user.


## Gene names should be changed as needed in the next cell.

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
Data=pd.read_csv('idModules.csv') # Define path to input file

def Slicedataframe():
    '''Define a function that slices the input dataframe into independent dataframes based on the Cluster names. Next, slice these dataframes based on the presence of the same motif, generating 'modules' '''  
    ClusterLST=Data['Cluster'].unique().tolist()                            # list of unique Cluster names (ie, 'Induced' and 'Repressed')
    lst=[]                                                                 
    DF=Data.copy()                                                         
    for cluster in ClusterLST:                                              # Select the first 'cluster' on the list 
        # Create a new dataframe by selecting only those rows that contain the selected 'cluster' in the 'Cluster' column 
        DF2=DF.loc[DF['Cluster']== cluster]                                 
        # From the newly created dataframe, place each instance of a unique motif into a list
        MotifLST=DF2['Motif'].unique().tolist()                             
        cleanedMotifLST = [x for x in MotifLST if str(x) != 'nan']          # drop the string 'nan' from the list. 'nan' occurs for Ppeps that did not have an identified Motif from Motif-X. 
        for motif in cleanedMotifLST:                                       # Select a motif in the list
            DF3=DF2.loc[DF['Motif']== motif]                                # Filter the dataframe, selecting only those rows that contain 'motif' in the Motif column
            DF3['freq'] = DF3.groupby('Motif')['Motif'].transform('count')  # Produce a new column, called 'freq' that contains the number of rows, and thus phospho-peptides, that contain a given motif.
            lst.append(DF3) 
    return lst

SlicedDF_lst=Slicedataframe()

def ConcatenateDFs():
    ''' Define a function that appends the dataframes in the SlicedDF_list together. '''
    EmptyDF = pd.DataFrame()                                                
    for df in SlicedDF_lst:                                                
        df=df.copy() 
        EmptyDF=EmptyDF.append(df)                                          # append to the empty DF the dataframe selected and overwrite the empty dataframe
    return EmptyDF

Final_DF=ConcatenateDFs()
FinalDFV2=Final_DF.fillna(0)                                                #  fill any NaN values with '0'

#--------------------------------------------------------------------------------------------------------------------------
def Module_Motif_NoMutantPhenotypeExists(df):
    ''' Define a function that assigns no-phenotype submodules'''
    if(df['ire1'] ==0) & (df['mkk1_2']==0):         #CHANGE GENE NAMES HERE
        return 'No_Phenotype_Exists'
    
FinalDFV2['Phenotype']=FinalDFV2.apply(Module_Motif_NoMutantPhenotypeExists, axis=1) 
FinalDFV2=FinalDFV2.loc[FinalDFV2['Phenotype']=='No_Phenotype_Exists']        # Select all rows for which "No_Phenotype_Exists" in the 'Phenotype' column.
FinalDFV2['subModule']=FinalDFV2.Cluster.map(str) + "_" + FinalDFV2.Motif + "_" + FinalDFV2.Phenotype                                   # create a new column, called submodule, that contains the concatenated strings in the 'Cluster', 'Motif', and 'Phenotype' columns.

# CHANGE GENE NAMES HERE
FinalDF=Final_DF.dropna(subset = ['ire1', 'mkk1_2'], how='all')        # Remove rows that have NaN in all 3 columns representing mutant phenotpes. This steps removes theNo-phenotype submodules which were creat                                                                               # ed above. 
FinalDF=FinalDF.fillna(0)                                              # fill any NaN that remain with '0'
lstCols=['ire1', 'mkk1_2']                                             # make a list that contains the column headers for the 3 mutants. 



def DefineMutantContribution(row):
    ''' Define a function that identifies for each phospho-peptide if it has a phenotype in more than one mutant strain'''
    dictData={} 
    for colname in lstCols:    
        if not row[colname]==0:                                                # if value is not equal to zero, there is a mutant phenotype (ex; Induced_defective)
            dictData[colname]=row[colname]  
    if len(dictData.keys())==0: return 0  
    else:
        return ":".join(dictData.keys())
    
FinalDF['Contribution']=FinalDF.apply(lambda x: DefineMutantContribution(x), axis=1) 



def DefinePhenotypeFromMutants(row):
    ''' Define a function that captures the mutant phenotype for Ppeps with multiple phenotypes and places it within a column'''
    dictData={}  
    for colname in lstCols:   
        if not row[colname]==0:
            dictData[colname]=row[colname] 
    if len(dictData.keys())==0: return 0 
    else:
        return ":".join(dictData.values()) 
    
FinalDF['Phenotype']=FinalDF.apply(lambda x: DefinePhenotypeFromMutants(x), axis=1)


#--------------------------------------------------------------------------------------------------------------------------------------------------
''' Determine all Ppeps that have 2 or more mutant phenotypes (ire1/mkk1_2 phenotypes), then the script produces a new column with individual subModule names for 
the ire1 phenotype''' 
# CHANGE GENE NAMES BELOW
FinalDF_multiplePhenotypes=FinalDF[FinalDF['Contribution'].str.contains(":")]     # Select 'contribution column rows that contain ":", which means the Ppep has two mutant phenotypes since this is a separator between gene names
FinalDF_multiplePhenotypes_ire1=FinalDF_multiplePhenotypes[FinalDF_multiplePhenotypes['Contribution'].str.contains("ire1")] 
FinalDF_multiplePhenotypes_ire1['Ire1']='ire1' 
FinalDF_multiplePhenotypes_ire1['subModule']=FinalDF_multiplePhenotypes_ire1.Cluster.map(str) + "_" + FinalDF_multiplePhenotypes_ire1.Motif + "_" + FinalDF_multiplePhenotypes_ire1.Ire1 + "_" + FinalDF_multiplePhenotypes_ire1.ire1

#--------------------------------------------------------------------------------------------------------------------------------------------------
''' Define all Ppeps that have 2 or more mutant phenotypes (ire1/mkk1_2 phenotypes), then the script produces a new column with individual subModule names for 
the mkk1_2 phenotype''' 
# CHANGE GENE NAMES
FinalDF_multiplePhenotypes_mkk1_2=FinalDF_multiplePhenotypes[FinalDF_multiplePhenotypes['Contribution'].str.contains("mkk1_2")]
FinalDF_multiplePhenotypes_mkk1_2['Mkk1_2']='mkk1_2'
FinalDF_multiplePhenotypes_mkk1_2['subModule']=FinalDF_multiplePhenotypes_mkk1_2.Cluster.map(str) + "_" + FinalDF_multiplePhenotypes_mkk1_2.Motif + "_" + FinalDF_multiplePhenotypes_mkk1_2.Mkk1_2 + "_" + FinalDF_multiplePhenotypes_mkk1_2.mkk1_2

#--------------------------------------------------------------------------------------------------------------------------------------------------
''' Define all Ppeps that have 2 or more mutant phenotypes (ire1/mkk1_2 phenotypes), then the script produces a new column with individual subModule names for 
the cdc14 phenotype'''
# CHANGE GENE NAMES, 
#FinalDF_multiplePhenotypes_cdc14=FinalDF_multiplePhenotypes[FinalDF_multiplePhenotypes['Contribution'].str.contains("cdc14")]
#FinalDF_multiplePhenotypes_cdc14['Cdc14']='cdc14'
#FinalDF_multiplePhenotypes_cdc14['subModule']=FinalDF_multiplePhenotypes_cdc14.Cluster.map(str) + "_" + FinalDF_multiplePhenotypes_cdc14.Motif + "_" + FinalDF_multiplePhenotypes_cdc14.Cdc14 + "_" + FinalDF_multiplePhenotypes_cdc14.cdc14

#--------------------------------------------------------------------------------------------------------------------------------------------------

'''This section of code appends the above mutant dataframes together (ie, FinalDF_multiplePhenotypes_cdc14, etc.) (contained ":"). The result is Ppeps with phenotypes in more than one strain are listed on multiple lines rather than a single line'''

FinalDF_mutants=FinalDF_multiplePhenotypes_ire1.append(FinalDF_multiplePhenotypes_ire1) 
FinalDF_mutants_Final=FinalDF_mutants.append(FinalDF_multiplePhenotypes_mkk1_2)


#CHANGE GENE NAMES,  if you have more than 2 gene names add them here after Peptide
FinalDF_mutants_Final=FinalDF_mutants_Final[['Ppep','Cluster','Motif','Peptide','ire1','mkk1_2','freq','Contribution','Phenotype','subModule']] # Only retain these columns 


#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Drop from the original dataframe rows containing Ppeps with multiple mutant phenotypes. 
FinalDF_minus_multiPhenotypePpeps=FinalDF[FinalDF.Contribution.str.contains(":")==False] # Removing all rows that contain ":", and thus are phospho-peptides with multiple mutant phenotypes


# Generate the final submodule names
FinalDF_minus_multiPhenotypePpeps['subModule']=FinalDF_minus_multiPhenotypePpeps.Cluster.map(str) + "_" + FinalDF_minus_multiPhenotypePpeps.Motif + "_" + FinalDF_minus_multiPhenotypePpeps.Contribution + "_" + FinalDF_minus_multiPhenotypePpeps.Phenotype


#-----------------------------------------------------------------------------------------------------------------------------------------------------
Ppeps_with_PhenotypesDF=FinalDF_minus_multiPhenotypePpeps.append(FinalDF_mutants_Final)  # Appending together the dataframes that originally had single mutant phenotypes, and the dataframe that started with multiple mutant Phentoypes, but now contains single listings for each Ppep-mutant phenotype



# Remove any submodule that only has a single Ppep constituent, since by default a submodule must contain 2 Ppeps. 
Ppeps_with_PhenotypesDF_subModules=Ppeps_with_PhenotypesDF[Ppeps_with_PhenotypesDF.duplicated(['subModule'], keep='last') | Ppeps_with_PhenotypesDF.duplicated(['subModule'])]  # only retain duplicates, get rid of single entries 



# Append to the dataframe with phenotype subModules, all No-Phenotype submodules
Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF=Ppeps_with_PhenotypesDF_subModules.append(FinalDFV2) # append to dataframe
Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF=Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF[['Ppep', 'Cluster', 'Motif', 'Peptide', 'ire1', 'mkk1_2', 'freq', 'Contribution', 'Phenotype', 'subModule']]

#-----------------------------------------------------------------------------------------------------------------------------------------------------
''' Create a column with the 'Module' name '''
Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF['Module']=Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF.Cluster.map(str) + "_" + Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF.Motif 

# Make a column concatenating Ppep and subModule and drop the duplicates based on that column
Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF['Name'] = Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF['Ppep'] + "_" + Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF['subModule']
Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF.drop_duplicates(subset='Name', keep='first', inplace=True)

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# define a function that will write out a dataframe as a tab separated file
def Dataframe_to_Tsv (dataframe, NewFileName):
    dataframe.to_csv (NewFileName,sep=',', index=False)

Dataframe_to_Tsv(Ppeps_with_Phenotypes_subModules_and_noPhenotypes_DF, 'Modules_pPep.csv') 
# The above file contains all modules and subModules with and without mutant phenotypes. 

print('Done')
# OUTPUT: Modules_pPep.csv
#   for the case of 2 genes ire1, mkk1_2, header looks like
#   Ppep    Cluster Motif   Peptide ire1    mkk1_2  freq    Contribution    Phenotype       subModule       Module

# Prep for Identify Shared Interactors 

input file : 
Submodule,ORF<br>
Induced_......TP....._cdc14_Repressed_Amplified,YLR319C<br>
Induced_......TP....._cdc14_Repressed_Amplified,YJL070C<br>

#### NOTE: ORF names have need to have the '-' removed,  YER074W-A becomes YER074A.

The other input files are provided.
'''

In [ ]:
##  CHANGE INPUT FILE HERE   ##
inputFile = 'Modules_pPep.csv'

# create the input file based on the output of the previous step.
with open('Submodule_constituents.csv', 'w') as out:
    out.write('Submodule,ORF\n')
    with open(inputFile,'r') as f:
        f.readline()                            # skip header
        for line in f:
            data = line.rstrip().split(',')    # CHECK THE FILE DELIMITER 
            name = data[0].split('_')[0]   
            name = re.sub('-', '', name)
            row  = data[9] + ',' + name + '\n'
            
            out.write(row)
out.close()
print('Done')
# OUTPUT: Submodule_constituents.csv

# Identify Shared Interactors

This script identifies proteins enriched for interactions with Submodule constituent proteins, based on known interactions in the background network. We call these proteins 'Shared Interactors'. The background network is a protein
interaction network curated in yeast under mostly nutrient replete conditions that contains 4638 proteins and ~ 25,000 interactions, including directed (ex; kinase-substrate), and 
non-directed. 

Proteins enriched for interactions with Submodule proteins at a 5% FDR, determined by a hypergeometric test and BH correction, are considered shared interactors.

Shared Interactors represent numerous functional classes, including kinases and phosphatases. Kinase and phosphatase shared interactors represent potential Submodule regulators.
 
HyperG function:
distrib=hypergeom(N,M,n)
distrib.pmf(m)

* N - population size (4638 unique proteins in Background network file - phospho_v4_bgnet_siflike_withdirections_Matt_Modified.csv)

* M - total number of successes  (# of interactions for a given protein. ie. Protein A has 200 known interactions in the background network).

* n - the number of trials (also called sample size) -  ie. (Number of proteins that reside within a submdoule)

* m - the number of successes - for example: Protein A, a shared interactor, has 35 interactions with proteins in Submodule B. 
 
 
 Final shared interactor file:   __Final_enriched.csv__  , this contains the significant Shared Interactors based on the
 BH_significance test.
 
 A list of all shared interactors can be found:  __Network_Submodule_Nodes_background_Network.csv__
 


In [ ]:
current_dir = os.getcwd()

Submodule_DF   = pd.read_csv(current_dir + '/Submodule_constituents.csv')   # File that contains Submodule names and their protein constituents
BgNet          = pd.read_csv(current_dir + '/SI_Identification_Input_Files/Background_Network.csv')                                                                                   # Background network of protein interactions
Num_Prot_Inter = pd.read_csv(current_dir + '/SI_Identification_Input_Files/Number_Interactions_Each_Protein.csv')                                              # Number of protein interactions for each protein in the background network
Annotation_DF  = pd.read_csv(current_dir + '/SI_Identification_Input_Files/Annotation.csv')                                                   # Yeast protein annotation file
 
Submodule_List=Submodule_DF['Submodule'].unique().tolist()                                                                                                  # Send the Submodules to a list, but filter out duplicates, which there will be many, since the Submodules will have been found in many proteins.

dicOrfs={}
for Submodule in Submodule_List:                                                                                                                            # Key (Submodule), Value (Yeast ORFs that are Submodule constituents). Filter ORFs found twice to single occurence (important for enrichment analysis)
    dicOrfs[Submodule]=(Submodule_DF.loc[Submodule_DF['Submodule'] == Submodule])['ORF'].unique().tolist()
        

dicOrfsCounts={}  
for k,v in dicOrfs.items():  
    if k not in dicOrfsCounts:  
        value=len(v)            
        dicOrfsCounts[k]=value
        
df_Submodule_Size=pd.DataFrame(list(dicOrfsCounts.items()),                                                                                                  # convert dict to dataframe.
                      columns=['Submodule','n'])

def SliceDataframe():
    ''' For each Submodule identify all proteins that interact with the Submodule proteins in the backgroudn network '''
    lst = []
    for key in dicOrfs.keys():                                                                                                                             #Select the key, which is a Submodule, from the dict
        CurrentDF=BgNet.copy() 
        x=CurrentDF[CurrentDF['Protein1'].isin(dicOrfs[key])].rename(columns={'Protein1':'Submodule_Containing_Proteins', 'Protein2':'Possible_Shared_Interactors'})                              #Create a new dataframe that is a slice of the salt background network, and only contains proteins that were passed in "dicOrfs[key]". At the same time, rename the columns                                
        x['Submodule']=key 
        lst.append(x)
        
    return lst

Sliced_dataframe_list= SliceDataframe()
      
def Add_n():    
    ''' Function adds 'n', the number of proteins in the Submodule, to each dataframe'''
    lst= []
    for df in Sliced_dataframe_list:
        NewDF=df.merge(df_Submodule_Size)
        lst.append(NewDF)
        
    return lst

Sliced_dataframe_list= Add_n()

def Identify_Shared_Interactors():
    ''' Function identifies proteins that interact with at least 2 protein constituents of each submodule'''
    
    lst=[] 
    for df in Sliced_dataframe_list: 
        NewDF=df.copy()
        NewDF2=NewDF[NewDF.duplicated(['Possible_Shared_Interactors'], keep = 'last')| NewDF.duplicated(['Possible_Shared_Interactors'])]                  # Only retain proteins that interact with at least 2 submodule protein constituents
        x=NewDF2.sort_values(by='Possible_Shared_Interactors', ascending=True) 
        lst.append(x)
       
    return lst

Shared_Interactors_lst=Identify_Shared_Interactors()

def AppendDFs_that_Contain_AllSharedInteractors_and_their_targets():
    ''' Function appends all submodules and their shared interactors together into a single file'''
    EmptyDF = pd.DataFrame() 
    for df in Shared_Interactors_lst:  
        df=df.copy() 
        EmptyDF=EmptyDF.append(df)
    return EmptyDF

SI_andTargets=AppendDFs_that_Contain_AllSharedInteractors_and_their_targets()

SI_andTargets_FINAL=pd.merge(left=SI_andTargets, right=Annotation_DF, how='left',
                              left_on='Possible_Shared_Interactors', right_on='systematic_name_dash_removed')                                               # complete a merge so I can get the dashes back in the names, which are not included in the background network
del SI_andTargets_FINAL['Possible_Shared_Interactors']                                                                                                      # drop because  lacks the dashes which are needed for the correct naming convention
del SI_andTargets_FINAL['systematic_name_dash_removed']                                                                                                     # drop because carried over from the merge
del SI_andTargets_FINAL['Directed']

SI_andTargets_FINAL.columns = ['Submodule_Containing_Proteins', 'Interaction', 'Submodule', 'n','Possible_Shared_Interactors']                        # rename columns

myDF = pd.DataFrame(SI_andTargets_FINAL)
# OUTPUT NAME FOR SHARED INTERACTORS
filename = 'SI_Identification_SubmoduleS__SIs_and_Targets_FDR.csv'
myDF.to_csv( filename, index=False, encoding='utf-8' )              # All interactions between SIs and their submodule constituent proteins. No enrichment at this step.

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
''' Preparing dataframe for Hypergeometric test'''

def Add_N_and_m():
    ''' Function adds 'N' and calculates 'm' values, which are inputs for the hypergeometric test, to the datframe'''
    lst=[]
    for df in Shared_Interactors_lst:
        NewDF=df.copy()
        NewDF['N'] = 4638          # THIS IS THE LENGTH OF THE DATA FRAME, *******************************                                                                                                                         # of proteins in the background network
        NewDF['m'] = NewDF.groupby('Possible_Shared_Interactors')['Possible_Shared_Interactors'].transform('count')
        lst.append(NewDF)
    
    return lst

Dataframes_list_with_n_N_m=Add_N_and_m()

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------


def Drop_dups():
    ''' For each dataframe, which contains a single submodule, it's protein constituents, and shared interactors, drop duplicate entries for identified SI proteins
    . This leaves a single entry for each shared interactor protein. '''
    lst=[]
    for df in Dataframes_list_with_n_N_m:
        NewDF=df.copy()
        Final_DF=NewDF.drop_duplicates('Possible_Shared_Interactors')
        Final_DF=Final_DF.rename(columns={'Possible_Shared_Interactors':'Shared_Interactor'})
        lst.append(Final_DF)
        
    return lst

Drop_Dups_lst=Drop_dups()


def Return_M():
    ''' Function identifies 'M' (the total number of interactions for each Shared Interactor protein in the background network) and adds that number
    to the dataframe'''
    lst=[]
    for df in Drop_Dups_lst:
        NewDF=df.copy()
        NewDF2=df.copy()
        NewDF_lst=NewDF['Shared_Interactor'].tolist()                                                                                                            # place all proteins in the 'Shared_Interactor' column in a list 
        Shared_Interactors=Num_Prot_Inter[Num_Prot_Inter['Protein'].isin(NewDF_lst)].rename(columns={'Protein':'Shared_Interactor', 'Total':'M'})
        Shared_Interactor_merge=Shared_Interactors.merge(NewDF2, on='Shared_Interactor')
        Shared_Interactor_merge=Shared_Interactor_merge.sort_values(by='Shared_Interactor', ascending=True)
        lst.append(Shared_Interactor_merge)
        
    return lst

Return_M_lst=Return_M()

#-----------------------------------------------------------------------------------------------------------------------------------------
def hyper(N,M,n,m): 
    ''' Function defines the parameters for a hypergeometric test that returns a p-value representing the chances of identifying >= x, where x is the number of successes '''  
    frozendist=hypergeom(N,M,n)
    ms=np.arange(m, min(n+1, M+1))
    rv=0;
    for single_m in ms: rv=rv+frozendist.pmf(single_m)
    return rv

def run_hyper():
    ''' Function calls the hypergeometric function above  on each shared interactor for each submodule'''
    lst=[]
    for df in Return_M_lst:
        if not df.empty:
            NewDF=df.copy()
            NewDF['p-value'] = NewDF.apply(lambda row: hyper(row['N'], row['M'], row['n'], row['m']), axis=1)
            lst.append(NewDF)
        
    return lst 

run_hyper_lst=run_hyper()

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
def AppendDFs():
    ''' Append DFs for each submodule and it's SIs together into a single DF'''   
    EmptyDF = pd.DataFrame() #
    for df in run_hyper_lst: 
        df=df.copy() 
        EmptyDF=EmptyDF.append(df)
    return EmptyDF

Final=AppendDFs()

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
''' Prepping for Benjamini Hochberg procedure. Below code is ranking p-values from 1 to n based on lowest to highest p-value score'''

Final=Final.sort_values(by=['p-value'],ascending=[True])                                                                                              # Sort p-values from lowest to highest
Final_resetIndex=Final.reset_index()                                                                                                        # Reset the index after the sort
Final_resetIndex.index +=1                                                                                                                  # start numbering at 1 for index
       
NewDF=Final_resetIndex
NewDF_Allp_values=Final_resetIndex
NewDF=NewDF[['p-value']]                                                                                                                    # select only the p-value column of the dataframe 
NewDF_dropdups=NewDF.drop_duplicates('p-value')                                                                                             # drop duplicate p-values
NewDF_dropdups=NewDF_dropdups.reset_index()                                                                                                 # reset the index
NewDF_dropdups.index +=1                                                                                                                    # start numbering at 1 for index
NewDF_dropdups['Rank(i)'] = NewDF_dropdups.index                                                                                            # #Add a rank column that will be filled with index values. 
NewDF_dropdups=NewDF_dropdups.drop('index', 1)                                                                                              # Drop the additional column 'index' that is not sorted.
NewDF_merge=NewDF_Allp_values.merge(NewDF_dropdups, on='p-value')                                                                           # create a new dataframe that is a merge of the dataframe with all p-values, and the dataframe with unique p-values and their ranks. 
NewDF_merge=NewDF_merge.drop('index',1)                                                                                                     # drop the index that was added from the merge. This leaves all p-values ordered from lowest to highest with their ranking.

'''Add parameters necessary for completing Benjamini-Hochberg procedure '''

NewDF=NewDF_merge
NewDF['m_(number_of_tests)']=(len(NewDF))                                                                                                   # Add 'm (number of tests)' column 
NewDF['Q_(FDR)']=0.05      # THIS IS THE FDR VALUE, USER CAN CHANGE **************************************                                                                                                                 # Add Q (FDR) column. This can be changed manually.
NewDF['(i/m)Q']=((NewDF['Rank(i)']/NewDF['m_(number_of_tests)'])*NewDF['Q_(FDR)'])                                                          # add the (i/m)Q column 
NewDF['BH_significant']=NewDF.apply(lambda x: 1 if x['p-value']<x['(i/m)Q'] else 0, axis=1)                                                 # Identify which proteins are  significant. 
NewDF=pd.merge(left=NewDF, right=Annotation_DF, how='left', left_on='Shared_Interactor', right_on='systematic_name_dash_removed')           # complete a merge to recover dashed version of YORFs
del NewDF['Shared_Interactor'] 
del NewDF['systematic_name_dash_removed']
del NewDF['Directed']
NewDF.columns = ['M','Submodule_Containing_Proteins', 'Interaction', 'Submodule', 'n','N','m','p-value','Rank(i)', 'm_(number_of_tests)', 'Q_(FDR)','(i/m)Q','BH_significant', 'Shared_Interactor'] # rename columns

myDF = pd.DataFrame(NewDF)
filename = 'Network_Submodule_Nodes_background_Network.csv'
myDF.to_csv( filename, index=False, encoding='utf-8' )       # Write out final file with enriched shared interactors for each submodule


# FILTER FOR THE FINAL Shared Interactors.
# Open and parse Network_Submodule_Nodes_background_Network.csv 
# Only keep the identified Shared Interactors about the first zero that appears in the BH_Significant column.
with open('Final_enriched.csv', 'w') as outfile, open('Network_Submodule_Nodes_background_Network.csv', 'r') as f:
    for line in f:
        if line.startswith('M,'):
            outfile.write(line)
            continue
        dat = line.split(',')
        if dat[12] == '0':
            break
        else:
            outfile.write(line)

f.close()
outfile.close()

# OUTPUT: SI_Identification_SubmoduleS__SIs_and_Targets_FDR.csv, 
#         Network_Submodule_Nodes_background_Network.csv
#         Final_enriched.csv

# Classify Shared Interactors Inputs Outputs

For each SI and it's connections with submodule protein constituents, determine if the SI acts upon the submodule (that is, the Shared Interactor has at least 1 directional interaction, or ppi interaction, with a submodule protein), or if the submodule acts upon the SI (that is, all interactions between the SI and submodule proteins have the 'Reverse' designation', indicating that the submodule proteins act upon the SI).

- If all of the interactions are reversed, then the script will define the relationship between the SI and the submodule as "Output"

- If there is at least one interaction that is directed from SI towards submodule, or is a ppi, the relationship between the SI and the submodule is defined as "Input"

This script takes an input file that contains the following:

- All enriched Shared Interactors (SIs) (according to HyperG) and their connections to submodules.
- All known protein interactions for each SI (ppi, kinase-substrate, etc)
- Many of these interactions are directed (kinase-substrate, metabolic pathway, etc). PPI are not a directed interaction.

Input: plain csv text file

Csv format:
SI_submodule,Shared_Interactor,SI_name,Motif_Containing_Proteins,submodule_Name
,Interaction_Directionality

YLR164C_Repressed_..RR.s.No_Phenotype_Exists,YLR164C,Tpk1,YDR207C,
Repressed_..RR.s.No_Phenotype_Exists, kinase_substrate

Column order is unimportant, column names must match above.

In [ ]:
import yeast_Gene_name_to_ORF as yg        

# create input files 
# SI_Identification_SubmoduleS__SIs_and_Targets_FDR.csv - input from ID shared interactors
submod = dict()
with open('SI_Identification_SubmoduleS__SIs_and_Targets_FDR.csv', 'r') as f:
    for line in f:
        row = line.rstrip().split(',')
        submod[row[0] + '_' + row[2]] = row

f.close()

# get network information
with open('classify_sharedInteractors_input.csv', 'w') as out:
    header = '%s,%s,%s,%s,%s,%s\n' %('SI_submodule','Motif_Containing_Proteins','SI_name','Shared_Interactor','submodule_Name'
,'Interaction_Directionality')
    out.write(header)
    with open('Final_enriched.csv') as f:
        for line in f:
            if line.startswith('M'):
                continue
            row = line.rstrip().split(',')
            if int(row[12]) != 1:
                continue
            name = row[1] + '_' + row[3]
            if name in submod:
                if row[1].endswith(('A','B')):
                    tmp = list(row[1])
                    tmp.insert(-1,'-')
                    row[1] = "".join(tmp)
                n = re.sub('-', '', row[1])
                ln = name + ',' + n + ',' + yg.sc_orfToGene[row[1]] + ',' + row[-1] + ',' + row[3] + ',' + row[2] + '\n'
                out.write(ln)

Input_df=pd.read_csv('classify_sharedInteractors_input.csv')

def Split_based_on_SI_submodule_Column():
    ''' Function splits the input DF into independent DFs based on the SI-submodule column pairs. Thus, each SI and it's submodule protein interactions are
    in independent dataframes '''
    DF_lst =[]
    for SI_submodule in Input_df['SI_submodule'].unique():
        DF=Input_df.loc[Input_df['SI_submodule']==SI_submodule]
        DF_lst.append(DF)
    return DF_lst

DF_lst=Split_based_on_SI_submodule_Column()

def Count_Instances_of_Reverse_Interaction():
    ''' Function counts, for each DF, and thus each SI-submodule pair, how many of the interactions are 'reversed', or facing from submodule TOWARDS SI. 
        It also counts the length of the dataframe, and then subtracts the the length of the dataframe from the counts. If the resultant value is 0, then all of the interactions 
        were reversed '''
    DF_Counts_lst=[]
    for df in DF_lst:
        df=df.copy()
        df['Counts']=df.Interaction_Directionality.str.contains('Reversed').sum()                                                               # Count the number of interactions that are "Reversed"
        x=len(df)
        df['Length']=x
        df['Counts_Length']=df['Counts']-df['Length']
        
        DF_Counts_lst.append(df)
    return DF_Counts_lst

DF_Counts_lst=Count_Instances_of_Reverse_Interaction()

def Only_Reverse_Interactions_Move_to_Outgoing_Columns():
    '''Function assigns 'Input' and 'Output' classifications based on the 'Counts_Length' column in the dataframe. '0' values are 'outputs', all other's are 'inputs' '''
    df_Modified_Outgoing_lst=[]
    for df in DF_Counts_lst:
        for value in df['Counts_Length'].unique():
           
            if value == 0:
                df['Shared_Interactor_submodule_Relationship']= 'Output'
                df_Modified_Outgoing_lst.append(df)
            else:
                df['Shared_Interactor_submodule_Relationship']= 'Input'
                df_Modified_Outgoing_lst.append(df)
           
    return df_Modified_Outgoing_lst
            
df_Modified_Outgoing_lst=Only_Reverse_Interactions_Move_to_Outgoing_Columns()


def AppendDFs(): 
    '''Function appends all dataframes back together '''
    EmptyDF = pd.DataFrame()
    for df in df_Modified_Outgoing_lst: 
        df=df.copy() 
        EmptyDF=EmptyDF.append(df) 
    return EmptyDF

Final=AppendDFs()    

Final_Keep_Columns_Needed_For_SIF=Final[['SI_submodule', 'Shared_Interactor', 'submodule_Name', 'Shared_Interactor_submodule_Relationship']]  
Final_Keep_Columns_Needed_For_SIF=Final_Keep_Columns_Needed_For_SIF.drop_duplicates('SI_submodule')                                                                     # Dropping duplicates entries, which are created because for each SI-submodule interaction there are numerous interactions with protein constituent. Only want a single interaction, input or output, for each SI and it's submodule. 

# create a new dataframe and write results to file
myDF = pd.DataFrame(Final_Keep_Columns_Needed_For_SIF)
filename = 'SIs_submodule_Relationships_Define_Network.csv'
myDF.to_csv(filename, index=False, encoding='utf-8',sep='\t') 
    
print('Done')
# OUTPUT:  classify_sharedInteractors_input.csv  -- DROP columns Motif_Containing Proteins Interaction_Directionality
#          SIs_submodule_Relationships_Define_ClassA_Network.csv